In [1]:
import os
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
from dotenv import load_dotenv
from src.utils import *
from src.utils_azure import *
load_dotenv('/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/official/.env')



endpoint = os.environ['AZURE_SHIBA_ENDPOINT_KEY']
subscription_key = os.environ['AZURE_SHIBA_SUBSCRIPTION_KEY']
COMPUTERVISION_CLIENT = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

/home/bu0ai/anaconda3/envs/shiba/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [2]:
image_path = '/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/data_label/sample-03.png'
det_draw_path = r'/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/demo/weight/draw_231229/best.pt'
det_code_path = r'/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/demo/weight/code_231207/best.pt'


det_draw_model = load_yolo(det_draw_path)
det_code_model = load_yolo(det_code_path)
raw_image_pil = load_image(image_path)
result_draw_df = yolo_inference(raw_image_pil, det_draw_model, type='draw')
raw_code_df = yolo_inference(raw_image_pil, det_code_model, type='code')
base_image_pil, draw_location_list = split_images(raw_image_pil.copy(), result_draw_df)
basic_info_dict = get_basic_info(base_image_pil, COMPUTERVISION_CLIENT)

/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/full_build_from_scratch_nhanhuynh/src/utils_azure.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_builder_name.sort_values(by='top', inplace=True)
/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/full_build_from_scratch_nhanhuynh/src/utils_azure.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_anken.sort_values(by='top', inplace=True)


In [3]:
code_df = raw_code_df[raw_code_df.boxes.apply(lambda b: calc_ovl(b, [803, 1169, 2478, 2944]) > 0.5)]
code_df.reset_index(drop=True, inplace=True)
num_cell, cell_width, cell_height = get_cell_size(code_df)
clear_cache_last_session('/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/full_build_from_scratch_nhanhuynh/tmpFile/tmp_core/')
clear_cache_last_session('/mnt/d/docker_volume/room_of_nhanhuynh/ai_butler/CAD-ShibaSangyo/full_build_from_scratch_nhanhuynh/tmpFile/tmp_gen_grid/')
concat_image_pil = gen_grid_image(raw_image_pil, code_df, num_cell, cell_width, cell_height)


Nothing cache to delete
Nothing cache to delete


In [5]:
draw_words, draw_boxes = ocr_azure_cad(concat_image_pil.copy(), COMPUTERVISION_CLIENT)
draw_boxes

[[5, 4, 61, 29],
 [374, 9, 404, 36],
 [1121, 13, 1134, 38],
 [194, 14, 234, 38],
 [564, 15, 583, 36],
 [742, 12, 765, 35],
 [919, 9, 947, 37],
 [18, 34, 51, 57],
 [206, 46, 225, 67],
 [383, 41, 408, 63],
 [567, 44, 582, 63],
 [737, 41, 763, 65],
 [924, 41, 953, 65],
 [1094, 45, 1162, 69],
 [15, 65, 50, 85],
 [185, 73, 241, 95],
 [367, 71, 421, 93],
 [546, 72, 599, 95],
 [728, 71, 781, 93],
 [907, 70, 965, 93],
 [6, 125, 61, 148],
 [184, 122, 239, 145],
 [386, 129, 412, 152],
 [565, 132, 592, 149],
 [730, 125, 770, 146],
 [922, 120, 978, 144],
 [1107, 127, 1131, 154],
 [21, 154, 48, 177],
 [199, 152, 226, 174],
 [390, 160, 407, 182],
 [564, 154, 613, 179],
 [737, 156, 768, 178],
 [908, 152, 1005, 178],
 [1109, 161, 1129, 182],
 [15, 184, 47, 206],
 [195, 182, 224, 203],
 [368, 189, 423, 212],
 [550, 187, 605, 211],
 [937, 183, 963, 206],
 [1092, 188, 1147, 212],
 [16, 247, 47, 268],
 [192, 246, 222, 266],
 [372, 244, 527, 270],
 [557, 241, 598, 274],
 [729, 240, 784, 264],
 [911, 242, 9

In [7]:
len(draw_boxes)

110

In [ ]:
extract_dict = mapping_code(raw_image_pil, draw_words, draw_boxes, code_df,
                                    num_cell, cell_width, cell_height)